<a href="https://colab.research.google.com/github/battuanudeep/BA-NUS-GAIP/blob/master/earthquake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from google.colab import drive
drive.mount ('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
df=pd.read_csv('/gdrive/My Drive/queryok.csv')

In [0]:
df=df.drop(['nst'],axis=1)
df=df.drop(['id'],axis=1)
df=df.drop(['updated'],axis=1)

In [0]:
df=df.dropna(subset = ['latitude', 'longitude'])

In [60]:
df[['Date','Time']] = df.time.str.split("T",expand=True)
df=df.drop(['time'],axis=1)
df[['year','month','day']] = df.Date.str.split("-",expand=True)
df=df.drop(['Date'],axis=1)
df[['time(hrs)','time(min)','time(sec)']] = df.Time.str.split(":",expand=True)
df=df.drop(['Time'],axis=1)
df['time(sec)'] = df['time(sec)'].map(lambda x: x.rstrip('Z'))
print(df)

       latitude  longitude   depth  mag  ... day  time(hrs)  time(min)  time(sec)
0       30.6505   131.0337   40.92  4.9  ...  19         20         28     41.575
1       40.6017   142.1060   54.79  5.1  ...  19         06         21     33.937
2       36.2728   140.8501   81.87  4.5  ...  19         00         23     40.577
3       43.4320   145.8823  110.76  4.7  ...  18         11         54     17.077
4       31.2373   130.3071  159.71  4.4  ...  17         20         17     19.817
...         ...        ...     ...  ...  ...  ..        ...        ...        ...
19331   42.9740   143.5680  126.80  4.1  ...  08         09         22     02.230
19332   36.3180   141.0900   47.80  4.1  ...  06         09         35     17.680
19333   44.4330   148.0160   90.10  4.7  ...  05         23         50     16.590
19334   45.0160   149.0060   33.00  4.3  ...  03         23         16     59.940
19335   44.3790   149.5160   33.00  5.4  ...  03         07         58     28.590

[19336 rows x 2

In [61]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).sum())

day                   0
depth              2201
depthError          877
dmin                 73
gap                2195
horizontalError     104
latitude              1
locationSource        0
longitude          2484
mag                1569
magError            173
magNst             1823
magSource             0
magType               0
month                 0
net                   0
place                 0
rms                 379
status                0
time(hrs)             0
time(min)             0
time(sec)             0
type                  0
year                  0
dtype: int64


In [0]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [63]:
df=remove_outlier(df, 'depth')
print(df.count())

latitude           17135
longitude          17135
depth              17135
mag                17135
magType            16620
gap                16622
dmin                4408
rms                16441
net                17135
place              17135
type               17135
horizontalError     3554
depthError         11454
magError            4280
magNst             13327
status             17135
locationSource     17135
magSource          17135
year               17135
month              17135
day                17135
time(hrs)          17135
time(min)          17135
time(sec)          17135
dtype: int64


In [64]:
print(df.isna().sum())
a = df['gap'].median(skipna=True)
b = df['rms'].median(skipna=True)
df['gap'] = df['gap'].replace(np.nan, a)
df['rms'] = df['rms'].replace(np.nan, b)
i = df[(df.status != 'reviewed')].index
df=df.drop(i, axis=0)
df=df.drop(['net','type','horizontalError','depthError','magError','magNst','status','locationSource','magSource','dmin'], axis=1)
df=remove_outlier(df, 'gap')
df=remove_outlier(df, 'rms')
print(df.isna().sum())
print(df.count())

latitude               0
longitude              0
depth                  0
mag                    0
magType              515
gap                  513
dmin               12727
rms                  694
net                    0
place                  0
type                   0
horizontalError    13581
depthError          5681
magError           12855
magNst              3808
status                 0
locationSource         0
magSource              0
year                   0
month                  0
day                    0
time(hrs)              0
time(min)              0
time(sec)              0
dtype: int64
latitude       0
longitude      0
depth          0
mag            0
magType      398
gap            0
rms            0
place          0
year           0
month          0
day            0
time(hrs)      0
time(min)      0
time(sec)      0
dtype: int64
latitude     14169
longitude    14169
depth        14169
mag          14169
magType      13771
gap          14169
rms          14169
pla

In [65]:
df.head()

,latitude,longitude,depth,mag,magType,gap,rms,place,year,month,day,time(hrs),time(min),time(sec)
0,30.6505,131.0337,40.92,4.9,mb,128.0,0.68,"9km SSE of Nishinoomote, Japan",2019,12,19,20,28,41.575
1,40.6017,142.1060,54.79,5.1,mww,82.0,0.84,"52km ENE of Hachinohe, Japan",2019,12,19,06,21,33.937
2,36.2728,140.8501,81.87,4.5,mb,140.0,1.01,"22km ESE of Oarai, Japan",2019,12,19,00,23,40.577
3,43.4320,145.8823,110.76,4.7,mb,133.0,0.70,"27km ENE of Nemuro, Japan",2019,12,18,11,54,17.077
5,30.7427,142.2374,10.00,4.4,mb,151.0,0.92,"Izu Islands, Japan region",2019,12,17,20,05,30.725
